In [ ]:
# 함수형 모델로 정의

# auto encoder 를 할 때 반드시 필요
# encoder와 decoder를 분리해야 하므로


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import models
from keras import layers

from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import backend as K

In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [5]:
tf.test.gpu_device_name()

!nvidia-smi

Tue Mar 23 07:54:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
(train_data, train_targets) , (X_test, y_test) = cifar10.load_data()

In [11]:
print(len(train_data))
print(train_data.shape)

print(len(train_targets))
print(train_targets.shape)
print(train_targets[:5])

50000
(50000, 32, 32, 3)
50000
(50000, 1)
[[6]
 [9]
 [9]
 [4]
 [1]]


In [9]:
print(len(X_test))
print(X_test.shape)

print(len(y_test))
print(y_test.shape)
print(y_test[:5])

10000
(10000, 32, 32, 3)
10000
(10000, 1)
[[3]
 [8]
 [8]
 [0]
 [6]]


## 2. Data Preprocessing

### Normalization

In [12]:
train_data = train_data.astype(float) / 255
X_test = X_test.astype(float) / 255

In [13]:
print(train_data)

[[[[0.23137255 0.24313725 0.24705882]
   [0.16862745 0.18039216 0.17647059]
   [0.19607843 0.18823529 0.16862745]
   ...
   [0.61960784 0.51764706 0.42352941]
   [0.59607843 0.49019608 0.4       ]
   [0.58039216 0.48627451 0.40392157]]

  [[0.0627451  0.07843137 0.07843137]
   [0.         0.         0.        ]
   [0.07058824 0.03137255 0.        ]
   ...
   [0.48235294 0.34509804 0.21568627]
   [0.46666667 0.3254902  0.19607843]
   [0.47843137 0.34117647 0.22352941]]

  [[0.09803922 0.09411765 0.08235294]
   [0.0627451  0.02745098 0.        ]
   [0.19215686 0.10588235 0.03137255]
   ...
   [0.4627451  0.32941176 0.19607843]
   [0.47058824 0.32941176 0.19607843]
   [0.42745098 0.28627451 0.16470588]]

  ...

  [[0.81568627 0.66666667 0.37647059]
   [0.78823529 0.6        0.13333333]
   [0.77647059 0.63137255 0.10196078]
   ...
   [0.62745098 0.52156863 0.2745098 ]
   [0.21960784 0.12156863 0.02745098]
   [0.20784314 0.13333333 0.07843137]]

  [[0.70588235 0.54509804 0.37647059]
   [0.6

### 2) One Hot Encoding

In [14]:
train_targets = to_categorical(train_targets)
y_test = to_categorical(y_test)

In [15]:
print(train_targets[:5])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_targets,
                                                      test_size = 0.2,
                                                      random_state = 2045)

## 3. Functional API Modeling

### 1) Model Define

In [24]:
%%time

input_img = layers.Input((32, 32, 3))

x = layers.Conv2D(filters = 32,
                  kernel_size = (3, 3),
                  strides = (1, 1),
                  padding = 'same')(input_img)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)


x = layers.Conv2D(filters = 32,
                  kernel_size = (3, 3),
                  strides = (2, 2),
                  padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)


x = layers.Conv2D(filters = 64,
                  kernel_size = (3, 3),
                  strides = (1, 1),
                  padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)


x = layers.Conv2D(filters = 64,
                  kernel_size = (3, 3),
                  strides = (2, 2),
                  padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(10)(x)
output_y_hat = layers.Activation('softmax')(x)

CIFAR = models.Model(input_img, output_y_hat)

CPU times: user 110 ms, sys: 1.13 ms, total: 111 ms
Wall time: 112 ms


In [25]:
CIFAR.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 16, 16, 32)        0   

In [26]:
CIFAR.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [27]:
Hist_CIFAR = CIFAR.fit(X_train, y_train,
                       epochs = 100,
                       batch_size = 128,
                       validation_data = (X_valid, y_valid))

Epoch 1/100
313/313 [==============================] - 38s 15ms/step - loss: 1.8977 - accuracy: 0.3636 - val_loss: 2.4433 - val_accuracy: 0.2009
Epoch 2/100
313/313 [==============================] - 4s 13ms/step - loss: 1.2351 - accuracy: 0.5608 - val_loss: 1.2496 - val_accuracy: 0.5582
Epoch 3/100
313/313 [==============================] - 4s 13ms/step - loss: 1.0334 - accuracy: 0.6333 - val_loss: 1.3943 - val_accuracy: 0.5296
Epoch 4/100
313/313 [==============================] - 4s 13ms/step - loss: 0.9202 - accuracy: 0.6760 - val_loss: 0.9686 - val_accuracy: 0.6567
Epoch 5/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8357 - accuracy: 0.7086 - val_loss: 1.0046 - val_accuracy: 0.6591
Epoch 6/100
313/313 [==============================] - 4s 13ms/step - loss: 0.7885 - accuracy: 0.7238 - val_loss: 0.8666 - val_accuracy: 0.6982
Epoch 7/100
313/313 [==============================] - 4s 13ms/step - loss: 0.7380 - accuracy: 0.7388 - val_loss: 0.9157 - val_accuracy

In [ ]:
loss, accuracy = CIFAR.evaluate(X_test, y_test)

print('Loss: {:.5f}' .format(loss))
print('Accuracy: {:.5f}' .format(accuracy))

## 4. 성능평가

### 1) Label Name 지정

In [ ]:
classes = np.array(['Airplane' , 'Automobile' , 'Bird' , 'Cat' , 'Deer' ,
                    ' Dog' , 'Frog' , 'Horse' , 'Ship' , 'Truck'])

preds = CIFAR.predict(X_test)
preds_single = classes[np.argmax(preds, axis = -1)]
actual_single = classes[np.argmax(y_test, axis = -1)]

### 2) 비교 시각화

In [ ]:
n_to_show = 10
indices = np.random.choice(range(len(X_test)), n_to_show)

fig = plt.figure(figsize = (15, 3))
fig.subplot_adjust(hspace = 0.4 , wspace = 0.4)

for i, idx in enumerate(indices) :
  img = X_test[idx]
  ax = fig.add_subplot(1, n_to_show, i + 1)
  ax.axis('off')

  ax.text(0.5, -0.35,
          'Pred = ' + str(preds_single[idx]),
          fontsize = 10,
          ha = 'center' ,
          transform = ax.transAxes)
  ax.test(0.5, -0.7,
          'Act = ' + str(actual_single[idx]),
          fontsize = 10,
          ha = 'center',
          transform = ax.transAxes)
  ax.imshow(img)